In [16]:
from langchain.document_loaders.url import UnstructuredURLLoader

In [17]:
from langchain.document_loaders.url import UnstructuredURLLoader

In [18]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
from dotenv import load_dotenv
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.document_loaders.url import UnstructuredURLLoader

from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores.faiss import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate


In [19]:
URLs=[
    "https://www.potentiam.co.uk/",
    "https://www.potentiam.co.uk/about-us",
    "https://blog.potentiam.co.uk/"
]

def DataLoader():
    loader=UnstructuredURLLoader(urls=URLs)
    data=loader.load()
    return data

In [20]:
data=DataLoader()

In [21]:

data

[Document(page_content='OFFSHORING PLUS+\n\nUnlock your full\n\npotential with a\n\nglobal team\n\nWe help you scale your business in a smarter way by building your offshore team\n\nSpeak to an advisor\n\nWHAT IS OFFSHORING PLUS+\n\nThe step-change for your business ambition and your people\n\nPotentiam is much more than an offshoring provider. We are both a business consultancy and an offshore service provider - focused squarely on realising your ambitions and the power of your people.\n\nWe partner with your management team to create Offshore Strategies that :\n\nEnables 3-5 x faster profitable growth\n\nDelivers significant free growth capital\n\nUnlocks your most valuable human resources\n\nLowers the risk of building new capabilities\n\nHow it works\n\nTHE POTENTIAM PROCESS\n\nHow it works\n\nOur experts help you plan, structure, build, integrate and operate outstanding international teams that allow your business and your in-house people to grow.\n\nStrategic Options\n\nOur strat

In [22]:
text_splitter=CharacterTextSplitter(separator='\n',
                                    chunk_size=500,
                                    chunk_overlap=50)

text_chunks=text_splitter.split_documents(data)

In [23]:
load_dotenv()
google_api_key=os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=google_api_key)

llm = ChatGoogleGenerativeAI(model="gemini-pro")

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [24]:
db = FAISS.from_documents(text_chunks, embeddings)
db.save_local("vectorstore")


In [25]:
custom_prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

prompt = PromptTemplate(template=custom_prompt_template,
                            input_variables=['context', 'question'])

qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type='stuff',
                                       retriever=db.as_retriever(search_kwargs={'k': 2}),
                                       return_source_documents=True,
                                       chain_type_kwargs={'prompt': prompt}
                                       )

In [26]:
query='tell me about Potentiam'
result=qa_chain.invoke(query)
print(result["result"])

Potentiam is a business consultancy and offshore service provider that helps businesses scale their operations by building offshore teams.


In [27]:
query='Who is the founder of Potentiam'
result=qa_chain.invoke(query)
print(result["result"])

Christopher Lydiard-Wilson


In [29]:
query='Name the cities where Potentiam is located'
result=qa_chain.invoke(query)
print(result["result"])

- Lași, Romania
- Cape Town, South Africa
- Bangalore, India
- London, UK


In [30]:
query='Potentiam is IT or non IT company'
result=qa_chain.invoke(query)
print(result["result"])

I don't know.


In [31]:
query='what is the buisness activities of potentiam'
result=qa_chain.invoke(query)
print(result["result"])

Potentiam is both a business consultancy and an offshore service provider, focused on realizing your ambitions and the power of your people.


In [33]:
query='Name the key peoples involved in building potentiam company'
result=qa_chain.invoke(query)
print(result["result"])

Christopher Lydiard-Wilson and Charles Fenton
